In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import re
import pymongo

In [2]:
def finding_raw_urls_from_base(url):
    service = Service("./chromedriver/chromedriver.exe")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
    
    driver = webdriver.Chrome(service=service, options=options)
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        EC.presence_of_all_elements_located((By.TAG_NAME, 'a'))
    )
    
    html = driver.page_source
    driver.quit()
    
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', href=True)
    raw_urls = []
    for link in links:
        if link['href'].startswith("https://www.moneycontrol.com/news/business/"):
            raw_urls.append(link['href'])
    return list(set(raw_urls))
    

In [3]:
# raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/")
raw_urls = finding_raw_urls_from_base("https://www.moneycontrol.com/news/business/stocks/page-2")

In [4]:
raw_urls

['https://www.moneycontrol.com/news/business/stocks/page-2/',
 'https://www.moneycontrol.com/news/business/markets/unhappy-with-cybersecurity-rules-pms-players-say-sebi-should-focus-on-outcome-not-methods-12906299.html',
 'https://www.moneycontrol.com/news/business/stocks/ok-play-india-shares-gain-nearly-5-ahead-of-jan-14-board-meeting-on-fundraise-12906442.html',
 'https://www.moneycontrol.com/news/business/stocks/page-6/',
 'https://www.moneycontrol.com/news/business/markets/tata-motors-stock-falls-3-despite-nomura-morgan-stanley-s-positive-jlr-outlook-12906230.html',
 'https://www.moneycontrol.com/news/business/stocks/',
 'https://www.moneycontrol.com/news/business/markets/borosil-renewables-hits-5-upper-circuit-for-3rd-day-announces-50-capacity-expansion-plans-12906286.html',
 'https://www.moneycontrol.com/news/business/stocks/page-4/',
 'https://www.moneycontrol.com/news/business/markets/religare-open-offer-mp-high-court-disposes-off-investor-petition-against-rbi-sebi-agm-can-now-

In [5]:
stock_urls =[]
market_urls = []
next_page_urls = []
other_urls = []
ipo_urls = []
def extract_urls(all_urls):
    for link in raw_urls:
        if link.startswith("https://www.moneycontrol.com/news/business/stocks/page"):
            next_page_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/stocks/"):
            stock_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/markets/"):
            market_urls.append(link)
        elif link.startswith("https://www.moneycontrol.com/news/business/ipo/"):
            ipo_urls.append(link)
        else:
            other_urls.append(link)

In [6]:
extract_urls(raw_urls)

In [7]:
stock_urls

['https://www.moneycontrol.com/news/business/stocks/ok-play-india-shares-gain-nearly-5-ahead-of-jan-14-board-meeting-on-fundraise-12906442.html',
 'https://www.moneycontrol.com/news/business/stocks/',
 'https://www.moneycontrol.com/news/business/stocks/gtpl-hathway-rises-up-to-17-ahead-of-q3-results-12906393.html',
 'https://www.moneycontrol.com/news/business/stocks/puravankara-stock-in-focus-as-firm-expands-land-portfolio-with-new-acquisition-in-bengaluru-12906483.html',
 'https://www.moneycontrol.com/news/business/stocks/neutral-manappuram-finance-target-of-rs-205-motilal-oswal-12906518.html',
 'https://www.moneycontrol.com/news/business/stocks/axis-bank-can-the-stock-reverse-underperformance-12906441.html',
 'https://www.moneycontrol.com/news/business/stocks/tcs-shares-fall-nearly-2-ahead-of-q3-results-12906508.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-09-2025-geojit-financial-services-12906340.html',
 'https://www.moneycontrol.com/news/bus

In [8]:

regex = r'https:\/\/www\.moneycontrol\.com\/news\/business\/stocks\/[^\/\s]+(?:\.[a-z]{2,6})(?:[\/\?].*)?'

final_stocks_urls = [url for url in stock_urls if re.match(regex, url)]

In [9]:
final_stocks_urls

['https://www.moneycontrol.com/news/business/stocks/ok-play-india-shares-gain-nearly-5-ahead-of-jan-14-board-meeting-on-fundraise-12906442.html',
 'https://www.moneycontrol.com/news/business/stocks/gtpl-hathway-rises-up-to-17-ahead-of-q3-results-12906393.html',
 'https://www.moneycontrol.com/news/business/stocks/puravankara-stock-in-focus-as-firm-expands-land-portfolio-with-new-acquisition-in-bengaluru-12906483.html',
 'https://www.moneycontrol.com/news/business/stocks/neutral-manappuram-finance-target-of-rs-205-motilal-oswal-12906518.html',
 'https://www.moneycontrol.com/news/business/stocks/axis-bank-can-the-stock-reverse-underperformance-12906441.html',
 'https://www.moneycontrol.com/news/business/stocks/tcs-shares-fall-nearly-2-ahead-of-q3-results-12906508.html',
 'https://www.moneycontrol.com/news/business/stocks/agri-picks-report-january-09-2025-geojit-financial-services-12906340.html',
 'https://www.moneycontrol.com/news/business/stocks/kalyan-jewellers-shares-extend-fall-on-fif

In [10]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["new_db"]
mycol = mydb['raw_news']

In [11]:
service = Service("./chromedriver/chromedriver.exe")
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.199 Safari/537.36")

driver = webdriver.Chrome(service=service, options= options)

In [12]:
def extract_data(url):  
    driver.get(url)
    
    WebDriverWait(driver, 60).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')    
    news = {}
    
    title = soup.find('h1', class_ = "article_title")
    news.update({"title":f"{title.text.strip()}"})
    
    desc = soup.find('h2', class_ = 'article_desc')
    news.update({"desc":f"{desc.text.strip()}"})
    
    date_time_div = soup.find('div', class_="article_schedule")
    if date_time_div:
        span_tag = date_time_div.find('span')
        date = span_tag.text.strip()
        news.update({"date":f"{date}"})
    
    datetime = date_time_div.text.strip()
    news.update({"datetime":f"{datetime}"})
    
    paragraphs_list = []
    paragrphs_div = soup.find('div', class_ ="content_wrapper")
    if paragrphs_div:
        paragraph_tags = paragrphs_div.find_all('p')
        for p in paragraph_tags:
            para_text = p.text.strip()            
            if len(para_text) < 50 :
                continue            
            if re.search(r"(click\s+here|disclaimer|modal|window|advertisement|investment\s+tips)", para_text, re.IGNORECASE):
                continue            
            paragraphs_list.append(para_text)         
            
    news.update({"content": paragraphs_list})
    
    stock_name = soup.find('a', class_="stock-name")
    if stock_name:
        news.update({"stock_name":f"{stock_name.text.strip()}"})
            
    # driver.quit()   
    return news

In [13]:
for url in final_stocks_urls :
    data = extract_data(url)
    x = mycol.insert_one(data)
    print(x.inserted_id)

6780ad2ec0ae801b14265326
6780ad4dc0ae801b14265327
6780ad52c0ae801b14265328
6780ad57c0ae801b14265329
6780ad5ac0ae801b1426532a
6780ad5ec0ae801b1426532b
6780ad67c0ae801b1426532c
6780ad70c0ae801b1426532d
6780ad75c0ae801b1426532e
6780ad7bc0ae801b1426532f
6780ad8fc0ae801b14265330
6780ad95c0ae801b14265331


In [14]:
driver.quit()   